In [1]:
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

%reload_ext autoreload
%autoreload 2

import time
import tracemalloc
from typing import Dict, List

import numpy as np
import polars as pl
import polars.selectors as cs
from wsri import somers_d_roc_auc, somers_d_two_pointers, wsri

## Global

In [2]:
rng: np.random.Generator = np.random.default_rng(seed=1227)

sample_sizes: np.typing.NDArray[np.int64] = np.array(
    [5_000, 50_000, 500_000, 5_000_000, 50_000_000]
)

score_column: str = "score"
label_column: str = "label"

## JIT Warmup

In [3]:
# Warm-up for JIT since the first call to a numba JIT function includes a one-time compilation overhead
warmup_sample_size: int = 1_000
benchmark: pl.DataFrame = pl.DataFrame(
    {
        score_column: rng.uniform(0, 1, warmup_sample_size).astype(np.float64),
        label_column: rng.choice([0, 1], warmup_sample_size).astype(np.int8),
    }
).lazy()

monitoring: pl.DataFrame = pl.DataFrame(
    {
        score_column: rng.uniform(0, 1, warmup_sample_size).astype(np.float64),
        label_column: rng.choice([0, 1], warmup_sample_size).astype(np.int8),
    }
).lazy()

wsri_value: float = wsri(
    benchmark=benchmark,
    monitoring=monitoring,
    score_column=score_column,
    label_column=label_column,
    quantiles=20,
    callback=somers_d_two_pointers,
)

print(f"WSRI: {wsri_value}")

WSRI: 4.687046070922117


## Wall Time

In [4]:
wall_time_results: List[Dict[str, float]] = []
data_sets: List[Dict[str, pl.LazyFrame]] = []

for sample_size in sample_sizes:
    print(f"Sample size: {sample_size}")

    benchmark: pl.LazyFrame = pl.DataFrame(
        {
            score_column: rng.uniform(0, 1, sample_size).astype(np.float64),
            label_column: rng.choice([0, 1], sample_size).astype(np.int8),
        }
    ).lazy()

    monitoring: pl.LazyFrame = pl.DataFrame(
        {
            score_column: rng.uniform(0, 1, sample_size).astype(np.float64),
            label_column: rng.choice([0, 1], sample_size).astype(np.int8),
        }
    ).lazy()

    data_sets.append({"benchmark": benchmark, "monitoring": monitoring})

    start_time_two_pointers: float = time.perf_counter()
    wsri_two_pointers: float = wsri(
        benchmark=benchmark,
        monitoring=monitoring,
        score_column=score_column,
        label_column=label_column,
        quantiles=20,
        callback=somers_d_two_pointers,
    )
    time_two_pointers: float = time.perf_counter() - start_time_two_pointers

    start_time_roc_auc: float = time.perf_counter()
    wsri_roc_auc: float = wsri(
        benchmark=benchmark,
        monitoring=monitoring,
        score_column=score_column,
        label_column=label_column,
        quantiles=20,
        callback=somers_d_roc_auc,
    )
    time_roc_auc: float = time.perf_counter() - start_time_roc_auc

    wall_time_results.append(
        {
            "sample_size": sample_size,
            "wsri_two_pointers": wsri_two_pointers,
            "wsri_roc_auc": wsri_roc_auc,
            "time_two_pointers": time_two_pointers,
            "time_roc_auc": time_roc_auc,
        }
    )

Sample size: 5000
Sample size: 50000
Sample size: 500000
Sample size: 5000000
Sample size: 50000000


## Memory

In [5]:
memory_results: List[Dict[str, float]] = []

for sample_size, data_set in zip(sample_sizes, data_sets):
    print(f"Sample size: {sample_size}")

    benchmark: pl.LazyFrame = data_set["benchmark"]
    monitoring: pl.LazyFrame = data_set["monitoring"]

    tracemalloc.start()
    _ = wsri(
        benchmark=benchmark,
        monitoring=monitoring,
        score_column=score_column,
        label_column=label_column,
        quantiles=20,
        callback=somers_d_two_pointers,
    )
    _, peak_mem_two_pointers = tracemalloc.get_traced_memory()
    tracemalloc.stop()

    tracemalloc.start()
    _ = wsri(
        benchmark=benchmark,
        monitoring=monitoring,
        score_column=score_column,
        label_column=label_column,
        quantiles=20,
        callback=somers_d_roc_auc,
    )
    _, peak_mem_roc_auc = tracemalloc.get_traced_memory()
    tracemalloc.stop()

    memory_results.append(
        {
            "sample_size": sample_size,
            "peak_mem_two_pointers_mb": peak_mem_two_pointers / 10**6,
            "peak_mem_roc_auc_mb": peak_mem_roc_auc / 10**6,
        }
    )

Sample size: 5000
Sample size: 50000
Sample size: 500000
Sample size: 5000000
Sample size: 50000000


## Combine Results


In [7]:
wall_time_data: pl.DataFrame = pl.from_dicts(wall_time_results)
memory_data: pl.DataFrame = pl.from_dicts(memory_results)

combined_results: pl.DataFrame = wall_time_data.join(
    other=memory_data, on="sample_size", how="inner"
).with_columns(
    (pl.col("time_roc_auc") / pl.col("time_two_pointers")).alias(
        "speedup_two_pointers_vs_roc_auc"
    ),
    (pl.col("peak_mem_roc_auc_mb") / pl.col("peak_mem_two_pointers_mb")).alias(
        "memory_two_pointers_vs_roc_auc"
    ),
)

combined_results.with_columns(cs.float().round(4))

sample_size,wsri_two_pointers,wsri_roc_auc,time_two_pointers,time_roc_auc,peak_mem_two_pointers_mb,peak_mem_roc_auc_mb,speedup_two_pointers_vs_roc_auc,memory_two_pointers_vs_roc_auc
i64,f64,f64,f64,f64,f64,f64,f64,f64
5000,-0.3375,-0.3375,0.0084,0.0091,0.1305,0.5022,1.0735,3.849
50000,1.5518,1.5518,0.0247,0.0256,1.2534,4.5781,1.0343,3.6526
500000,-0.3074,-0.3074,0.1923,0.2456,12.5036,45.0777,1.2776,3.6052
5000000,0.868,0.868,2.2323,2.7045,125.0035,450.0781,1.2115,3.6005
50000000,-0.529,-0.529,25.5728,31.567,1250.004,4500.0779,1.2344,3.6001
